I used https://www.kaggle.com/masonblier/aerial-cactus-simple-cnn, https://towardsdatascience.com/image-classification-python-keras-tutorial-kaggle-challenge-45a6332a58b8, and https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py to help get me started. 

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import imageio
from random import shuffle
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator




Using TensorFlow backend.


In [2]:
def label_img(name):
    word_label = name.split('-')[1]
    if word_label == 'Cactus': return 1
    elif word_label == 'NoCactus' : return 0

In [3]:

def load_training_data():
    train_load = []
    
    for img in os.listdir('../input/labeled-training-images/labeled_train/labeled_train'):
        strippedName = img.split('.')[0]
        label = label_img(strippedName)
        path = os.path.join('../input/labeled-training-images/labeled_train/labeled_train', img)
        if "DS_Store" not in path:
            img = Image.open(path)
            train_load.append([np.array(img), label])
           
            
    shuffle(train_load)
    return train_load

In [4]:
train_data = load_training_data()

trainImage = np.array([i[0] for i in train_data]).reshape(-1, 32, 32, 3)
trainLabels = np.array([i[1] for i in train_data])
trainLabels = keras.utils.to_categorical(trainLabels, 2)
trainImage  = trainImage/255

In [5]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    #rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=.15)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()#rescale=1. / 255)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())


model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# train_generator = train_datagen.flow_from_directory(
#     '../input/dataforaugmentation/labeled_train_augment/labeled_train_augment',
#     target_size=(32, 32),
#     batch_size=16,
#     class_mode='binary')
train_datagen.fit(trainImage)

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [9]:
#model.fit(trainImage, trainLabels, epochs = 15, verbose = 1, validation_split=.15)
#model.fit_generator(train_generator, steps_per_epoch=17500//16, verbose=1)
model.fit_generator(train_datagen.flow(trainImage, trainLabels, batch_size=32), steps_per_epoch=len(trainImage)/32, epochs=75)

Instructions for updating:
Use tf.cast instead.
Epoch 1/75
547/546 [==============================] - 18s 32ms/step - loss: 0.2338 - acc: 0.9081
Epoch 2/75
547/546 [==============================] - 12s 21ms/step - loss: 0.1216 - acc: 0.9531
Epoch 3/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0844 - acc: 0.9695
Epoch 4/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0647 - acc: 0.9766
Epoch 5/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0613 - acc: 0.9777
Epoch 6/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0615 - acc: 0.9767
Epoch 7/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0534 - acc: 0.9811
Epoch 8/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0495 - acc: 0.9822
Epoch 9/75
547/546 [==============================] - 12s 21ms/step - loss: 0.0545 - acc: 0.9813
Epoch 10/75
547/546 [==============================] - 12s 21ms/step - loss: 0.

In [10]:
def load_test_data():
    test_data = []
    names =[]
    for img in os.listdir("../input/aerial-cactus-identification/test/test"):
        name = img
        path = os.path.join("../input/aerial-cactus-identification/test/test", img)
        if "DS_Store" not in path:
            img = Image.open(path)
            test_data.append([np.array(img)])
            names.append(name)
            
    
    return (test_data, names)

In [11]:

(test_data, names) = load_test_data()

testImages = np.array([i[0] for i in test_data]).reshape(-1, 32, 32, 3)
testImages = testImages/255

In [12]:
results = model.predict_classes(testImages)

In [13]:

df = pd.DataFrame({'id':names,'has_cactus': results})
df.index.name='id'

df.to_csv("keras_results.csv", index=False)